# Build and Train Decision Trees for Classification Problems
## Purpose
In this example we will demonstrate how to:
   - Build a coreset tree for classification decision trees based on a real-world multiclass dataset.
   - Perform coreset tree vs. entire train dataset vs. random selection comparison as follows:
      - Retrieve the coreset from the coreset tree using get_coreset, and train an sklearn's GradientBoostingClassifier model on it.
      - Build an GradientBoostingClassifier model on the entire train dataset.
      - Build an GradientBoostingClassifier model on a randomly-sampled train dataset, where the sample size exactly matches coreset size.
      - Build an GradientBoostingClassifier model on a randomly-sampled train dataset, where the sample size is large-enough to strive to match the quality of using the coreset from the coreset tree.
      - Compare the quality of all four models built.
   - Repeat the comparison sets above, using two additional flavours:
      - Utilizing GradientBoostingClassifier sklearn classifier.
      - Calling our library's service object's direct fit function (employing GradientBoostingClassifier behind the scenes).
   - Employ coreset tree's utility functions for saving, loading and plotting the coreset tree.

For the example dataset, we will be using the PokerHand OpenML dataset (https://www.openml.org/search?type=data&status=active&id=155).
The data will be split into 'train' and 'test' portions using sklearn's train/test split (80% for the train portion, 20% for the test).
The 'train' portion will consist of roughly 663K samples, and the 'test' portion will have around 166K samples.
Our comparisons will utilize the balanced accuracy (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html) as the evaluation metric.


In [1]:
import os
import warnings
from time import time

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


from dataheroes import CoresetTreeServiceDTC

warnings.simplefilter('ignore', DeprecationWarning)

## 1. Prepare the dataset & the testing environment

In [2]:
# Load PokerHand dataset as X, y (data, target) from openml.
X, y = fetch_openml("pokerhand", return_X_y=True)

# Label-Encode the target. We do so because, further down the road, we will use GradientBoostingClassifier for
# modeling, which requires consecutive integer labels starting from 0.
label_enc = LabelEncoder()
y = label_enc.fit_transform(y)

# Merge 3 class labels into a single label, to make the data-set have only 5 targets.
# (Otherwise, the smaller class is made up of only 2 samples, and the last but one - with only 11.)
y[y > 3] = 4

# Apply standard scaling on the X.
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split to train and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=62)

# Remember the number of classes (5) required for the upcoming training code.
num_classes = len(np.unique(y_train))
print(f"#classes={num_classes}, train size={len(X_train):,}, test size={len(X_test):,}")

# Define the number of estimators for GradientBoostingClassifier.
# We set the number of estimators to 50 because GradientBoostingClassifier is a slow learner and for the sake of time we 
# reduce the number of estimators. 
# Note that in this case the scores will be lower than if we had used a higher number of estimators.
# The higher the number of estimators, the better the results will be (but the longer the time will take).
n_estimators = 50

# Set the tree access level which we will be working with (default is 0=root node).
tree_query_level = 2

# Remember the number of samples for the full dataset.
n_samples_full = len(y_train)

# Define the number of samples for the comparison flavour in which a randomly-selected dataset is striving to resemble coreset dataset's quality.
n_samples_rand_large = int(n_samples_full * 0.3)

# Define method for summarizing every flavour's experimental results in one neat table.
def produce_results(experiment_group : str,
                    n_samp_full : int, n_samp_rand_large : int, n_samp_coreset : int,
                    full_score : float, rand_large_score : float, rand_csize_score : float, coreset_score : float,
                    full_secs : float, rand_large_secs : float, rand_csize_secs : float, coreset_secs : float):

    df = pd.DataFrame(data={
        ' ': ['Full dataset', 'Random bigger-sized sample', 'Random smaller-sized sample', 'Coreset'],
        'Training dataset size': [n_samp_full, n_samp_rand_large, n_samp_coreset, n_samp_coreset],
        '% of full dataset': [n_samp_full / n_samp_full, n_samp_rand_large / n_samp_full, n_samp_coreset / n_samp_full, n_samp_coreset / n_samp_full],
        'Balanced accuracy score': [full_score, rand_large_score, rand_csize_score, coreset_score],
        'Training time (sec)': [full_secs, rand_large_secs, rand_csize_secs, coreset_secs],
    })
    last_row = pd.IndexSlice[df.index[-1], :]
    styles = [dict(selector="caption", props=[("text-align", "center"), ("font-size", "120%"), ("font-weight", "bold")])]
    s  = df.style \
        .set_properties(subset=[' '],**{'text-align':'left'}) \
        .set_properties(subset=['Training dataset size','% of full dataset','Balanced accuracy score','Training time (sec)'],**{'text-align':'right'}) \
        .set_properties(subset=last_row, **{'color':'green', 'font-weight':'bold'}) \
        .set_caption(f"{experiment_group} Results") \
        .format({
        'Training dataset size': '{:,}',
        '% of full dataset': '{:.2%}',
        'Balanced accuracy score': '{:.4f}',
        'Training time (sec)': '{:.2f}'}) \
        .hide(axis='index') \
        .set_table_styles(styles)
    return s

#classes=5, train size=663,360, test size=165,841


## 2. Build the coreset tree
Create a new service object and run `build` directly on the X, y train data.
We pass `chunk_size` and `coreset_size` instead of passing `n_instances` to initialize the coreset tree.

In [3]:
t = time()
# Define all columns as categorical, so they would be one-hot encoded by the dataheroes library.
data_params = {
    'categorical_features': [i for i in range(X_train.shape[1])],
    'ohe_min_frequency': 0
}
# We explicitly set the model to GradientBoostingClassifier, as the default
# model class is XGBClassifier in case XGBoost is installed
service_obj = CoresetTreeServiceDTC(optimized_for='training',
                                    chunk_size=40_000,
                                    coreset_size=20_000,
                                    data_params=data_params,
                                    model_cls=GradientBoostingClassifier,
                                   )
service_obj.build(X_train, y_train,copy=True)
coreset_build_secs = time() - t
print(f"Coreset build time (sec): {coreset_build_secs:.2f}")

Coreset build time (sec): 12.30


## 3. Tree-based coreset performance comparison: GradientBoostingClassifier modeling approach

### 3.1 Get a coreset from the tree & train a model based on it

Get the coreset from the tree using the ‘auto’ preprocessing_stage, which returns the data after applying auto-preprocessing, including one-hot-encoding. 


In [4]:
t = time()
coreset = service_obj.get_coreset(level=tree_query_level, preprocessing_stage='auto')
indices_coreset_, X_coreset, y_coreset = coreset['data']
w_coreset = coreset['w']

# Train an GradientBoostingClassifier model on the coreset.
GradientBoostingClassifier_coreset_model = GradientBoostingClassifier(
                                            n_estimators=n_estimators)
GradientBoostingClassifier_coreset_model.fit(X_coreset, y_coreset, sample_weight=w_coreset)
n_samples_coreset = len(y_coreset)
GradientBoostingClassifier_coreset_secs = time() - t

### 3.2. Train a model on the entire dataset (for comparison)
We use the same (entire) training part of the dataset that was used for building the coreset tree.

In [5]:
# One-hot encode all columns (not only the categorical ones).
# We may, in fact, encode only categorical columns - but we do the inclusive encoding
# here, since even the numeric features have small number of distinct values.
one_hot = OneHotEncoder(sparse=False)
one_hot.fit(X_train)
X_train = one_hot.transform(X_train)
X_train = X_train.astype(np.int8)
# save original X_test
X_test_original = X_test.copy()
# same preparation for X_test
X_test = one_hot.transform(X_test)
X_test = X_test.astype(np.int8)

In [6]:
t = time()
# Train an GradientBoostingClassifier model on the entire data-set.
GradientBoostingClassifier_full_model = GradientBoostingClassifier(
                                         n_estimators=n_estimators)
GradientBoostingClassifier_full_model.fit(X_train, y_train)
GradientBoostingClassifier_full_secs = time() - t

### 3.3. Train models on samples selected randomly from the training set

### 3.3.1. Produce a randomly-selected sample matching the coreset's size

In [7]:
t = time()
# Train an GradientBoostingClassifier model on the random data-set -
# (1) Create the model, (2) randomly sample the exact size as the coreset size, and (3) train the model.
GradientBoostingClassifier_rand_csize_model = GradientBoostingClassifier(
                                               n_estimators=n_estimators)
GradientBoostingClassifier_rand_csize_idxs = np.random.choice(n_samples_full, n_samples_coreset, replace=False)
GradientBoostingClassifier_rand_csize_model.fit(X_train[GradientBoostingClassifier_rand_csize_idxs, :], y_train[GradientBoostingClassifier_rand_csize_idxs])
GradientBoostingClassifier_rand_csize_secs = time() - t

### 3.3.2. Produce a randomly-selected sample striving to match coreset's quality

In [8]:
t = time()
# Train an GradientBoostingClassifier model on the random data-set -
# (1) Create the model, (2) randomly sample size larger than the coreset size, and (3) train the model.
GradientBoostingClassifier_rand_large_model = GradientBoostingClassifier(
                                               n_estimators=n_estimators)
GradientBoostingClassifier_rand_large_idxs = np.random.choice(n_samples_full, n_samples_rand_large, replace=False)
GradientBoostingClassifier_rand_large_model.fit(X_train[GradientBoostingClassifier_rand_large_idxs, :], y_train[GradientBoostingClassifier_rand_large_idxs])
GradientBoostingClassifier_rand_large_secs = time() - t

### 3.4. Compare models quality
Test coreset, random samples and full models on a test dataset, and compare their evaluation scores.

In [9]:
# Evaluate models.
GradientBoostingClassifier_full_score = balanced_accuracy_score(y_test, GradientBoostingClassifier_full_model.predict(X_test))
GradientBoostingClassifier_rand_large_score = balanced_accuracy_score(y_test, GradientBoostingClassifier_rand_large_model.predict(X_test))
GradientBoostingClassifier_rand_csize_score = balanced_accuracy_score(y_test, GradientBoostingClassifier_rand_csize_model.predict(X_test))
# We call auto_preprocessing on the test data, to prepare it in the same way
# the train data was processed and was used to build the model.
GradientBoostingClassifier_coreset_score = balanced_accuracy_score(y_test, GradientBoostingClassifier_coreset_model.predict(service_obj.auto_preprocessing(X=X_test_original, copy=True)['data']))
produce_results("GradientBoostingClassifier",
                n_samples_full, n_samples_rand_large, n_samples_coreset,
                GradientBoostingClassifier_full_score, GradientBoostingClassifier_rand_large_score, GradientBoostingClassifier_rand_csize_score, GradientBoostingClassifier_coreset_score,
                GradientBoostingClassifier_full_secs, GradientBoostingClassifier_rand_large_secs, GradientBoostingClassifier_rand_csize_secs, GradientBoostingClassifier_coreset_secs)

,Training dataset size,% of full dataset,Balanced accuracy score,Training time (sec)
Full dataset,"663,360",100.00%,0.4959,577.40
Random bigger-sized sample,"199,008",30.00%,0.4881,153.59
Random smaller-sized sample,"43,968",6.63%,0.4735,16.71
Coreset,"43,968",6.63%,0.5868,17.73


## 4. Tree-based coreset performance comparison: coreset tree's default fit modeling approach

### 4.1 Train a model directly on the coreset tree service object

In [10]:
t = time()
# Ignore GradientBoostingClassifier's warning about the requirement to pass the 'sample_weight' 
# as a keyword arg and not as a positional arg.
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Please note the direct usage of "service_obj.fit()".
# Provide the same parameters to the fit method as you'd provide for the model generation and 
# for tree access (required tree level).
service_obj.fit(level=tree_query_level,
                n_estimators=n_estimators
               )
service_obj_n_samples_coreset = len(service_obj.get_coreset(level=tree_query_level)['w'])
service_obj_coreset_secs = time() - t

Using GradientBoostingClassifier model. To use a different model, please set the model parameter. E.g.: service_obj.fit(model=LGBMClassifier(**params), **other_params) or set the model_cls during init or use the set_model_cls function


### 4.2. Train a model on the entire dataset (for comparison)
We use the same (entire) training part of the dataset that was used for building the coreset tree.
(Please note that for the sake of time saving, we reuse the same exact model generated in the step 3.2. above.)

In [11]:
# Just assign the same model and execution time, and do not call `fit`, since the model has already been trained.
service_obj_full_model = GradientBoostingClassifier_full_model
service_obj_full_secs = GradientBoostingClassifier_full_secs

## 4.3. Train models on samples selected randomly from the training set



In [12]:
# Just assign the same model and execution time, and do not call `fit`, since the model has already been trained.
service_obj_rand_csize_model = GradientBoostingClassifier_rand_csize_model
service_obj_rand_csize_secs = GradientBoostingClassifier_rand_csize_secs

### 4.3.2. Produce a randomly-selected sample striving to match coreset's quality
(Please note that for the sake of time saving, we reuse the same exact model generated in the step 3.3.2. above.)

In [13]:
# Just assign the same model and execution time, and do not call `fit`, since the model has already been trained.
service_obj_rand_large_model = GradientBoostingClassifier_rand_large_model
service_obj_rand_large_secs = GradientBoostingClassifier_rand_large_secs

### 4.4. Compare models quality
Test coreset, random samples and full models on a test dataset, and compare their evaluation scores.

In [22]:
# Evaluate models. Please note the direct usage of "service_obj.predict()".
service_obj_full_score = balanced_accuracy_score(y_test, service_obj_full_model.predict(X_test))
service_obj_rand_large_score = balanced_accuracy_score(y_test, service_obj_rand_large_model.predict(X_test))
service_obj_rand_csize_score = balanced_accuracy_score(y_test, service_obj_rand_csize_model.predict(X_test))
# pass original data to predict, due to its build-in preprocessing
service_obj_coreset_score = balanced_accuracy_score(y_test, service_obj.predict(X_test_original,copy=True))
produce_results("Service Object (Default)",
                n_samples_full, n_samples_rand_large, service_obj_n_samples_coreset,
                service_obj_full_score, service_obj_rand_large_score, service_obj_rand_csize_score, service_obj_coreset_score,
                service_obj_full_secs, service_obj_rand_large_secs, service_obj_rand_csize_secs, service_obj_coreset_secs)

,Training dataset size,% of full dataset,Balanced accuracy score,Training time (sec)
Full dataset,"663,360",100.00%,0.4959,577.40
Random bigger-sized sample,"199,008",30.00%,0.4881,153.59
Random smaller-sized sample,"43,968",6.63%,0.4735,16.71
Coreset,"43,968",6.63%,0.5868,17.40


## 5. Tree utilities

### 5.1. Save the tree for future use

In [ ]:
out_dir = Path('output')
out_dir.mkdir(exist_ok=True)
save_tree_name = 'train_coreset_tree'
service_obj.save(out_dir, save_tree_name, override=True)

### 5.2. Load the saved tree

In [ ]:
service_obj = CoresetTreeServiceDTC.load(out_dir, save_tree_name)

### 5.3. Plot the tree
Plot the tree to show the loading from files was properly done and to visualize the way the data is structured inside it.

In [ ]:
output_dir = Path('output')
output_dir.mkdir(exist_ok=True)
service_obj.plot(output_dir)

### 4.4. Save coreset for future use

In [ ]:
final_coreset_path = os.path.join(output_dir, "final_coreset")
service_obj.save_coreset(final_coreset_path)